# ChatBot Project
This Chatbot project is intended for Human AI Class of University of Nottingham. Expect a fully running code with some markdowns to make it clear.

*Disclaimer: This project uses Virtual Environment. Please download the requirements before running the program*

## Program Name: Costa Coffee ChatBot

The program is intended to take orders from customer, with a Natural Language (in English) as the input of the order.

# JSON File

`intents.json` is a JSON file containing the list of intents both for general talk and to take orders. This will serve as a decision tree for further processesing. 

In [16]:
# Get the data from JSON
import json

intentFile = open('./data/intents.json').read()
intents = json.loads(intentFile)

In [17]:
# Download stopwords from NLTK
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = stopwords.words('english')

# Remove negation
stopwords.remove('not')

[nltk_data] Downloading package stopwords to /Users/jr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
# Import libraries
import nltk
from nltk.tokenize import word_tokenize
import string

# Preprocess the intent
def preprocess(textinput):
    textinput = textinput.lower()

    # Remove punctuations
    string.punctuation = string.punctuation + '-'
    text_filtered = ''.join([char for char in textinput if char not in string.punctuation])

    # Tokenize the word
    text_tokens = word_tokenize(text_filtered)

    # Remove stopword from the text
    tokens_without_sw = [word for word in text_tokens if word.lower() not in stopwords]

    # Removing any stopwords from the sentence
    filtered_sentence = (" ").join(tokens_without_sw)

    post = nltk.pos_tag(word_tokenize(filtered_sentence), tagset='universal') 
    return post

In [21]:
# Intent reading with spell checker
import json
import difflib

def getIntent(preprocessedText):
    intentsList = []
    for i in preprocessedText:
        for j in range(len(intents['classes'])):
            a = difflib.get_close_matches(i[0], intents['classes'][j]['keyword'])
            if len(a) == 1:
                intentsList.append([intents['classes'][j]['rank'], intents['classes'][j]['class']])
            else:
                pass
    return sorted(intentsList)

In [22]:
a = 'i want to drinking a latt'
a_pre = preprocess(a)
print(getIntent(a_pre))

TypeError: list.append() takes exactly one argument (2 given)